In [1]:
import pandas as pd
import numpy as np
import json
import gzip
import math
import random
import pickle as pkl
from datetime import datetime
from collections import Counter
#import matplotlib.pyplot as plt
#import seaborn as sns
import os
from os.path import join
import ujson as json
import re
from multiprocessing import Pool

import ujson as json
import pandas as pd
import pycld2
import pickle as pkl


In [2]:
os.chdir('/shared/3/projects/style-influence/')

## Walk

In [ ]:
### PANDAS

subreddits_of_interest = set(['AmItheAsshole', 'DebateAChristian', 'DebateAnAtheist'])

colnames = ['body', 'author','comment_id','created_utc',
            'controversiality','score',
            'author_flair_css_class','author_flair_text',
            'depth','max_depth','subreddit','root_id',
            'parent_author','parent_id','parent_created_utc',
            'parent_controversiality','parent_score',
            'parent_flair_css_class','parent_flair_text']

from nltk.tokenize import regexp_tokenize
def tokenizebody(x):
    x = re.sub('\\bhttp.*\\b','',x.lower())
    return regexp_tokenize(x, '\w+')

import liwc
parse, category_names = liwc.load_token_parser('/shared/3/projects/agrimaTwitter/LIWC2015_English.dic')
def liwc_count(list_tokens):
    liwc_count = Counter(category for token in list_tokens for category in parse(token))
    liwc_count['length'] = len(list_tokens)
    return liwc_count

import regex
RE_BAD_CHARS = regex.compile(r"[\p{Cc}\p{Cs}]+")
def remove_bad_chars(text):
    return RE_BAD_CHARS.sub("", text)

def build_df(all_threads, outfile, counter):
    df = pd.DataFrame(all_threads, columns=colnames)
    df = df[df['author']!=df['parent_author']]
    df['tokens'] = df['body'].astype(str).apply(tokenizebody)
    df['tokens_liwc'] = df['tokens'].apply(liwc_count)
    df = df.drop_duplicates(subset=['comment_id'])
    df = pd.merge(df, df[['comment_id','tokens','tokens_liwc']], 
                  suffixes=('', '_parent'),
                  left_on='parent_id', right_on='comment_id', how='left')
    padding = ''
    if counter <= 9: padding = '0'
    pkl.dump(df, 
             open(outfile+'-'+padding+str(counter)+'.pkl','wb'))
    pkl.dump(df[df.subreddit.isin(subreddits_of_interest)], 
             open(re.sub('-aba','-subreddits',outfile)+'-'+padding+str(counter)+'.pkl','wb'))
    pkl.dump(df[df.depth > 0].sample(n=min(75000,df.shape[0])), 
             open(re.sub('-aba','-random',outfile)+'-'+padding+str(counter)+'.pkl','wb'))
    #df.to_csv(open(outfile+'-'+str(counter)+'.tsv','wb'), sep='\t', index=None)
    return df.groupby(['subreddit','author'])['comment_id'].count().reset_index()

langs = set(['en','un'])
def parse_comment(item, parent=['','','','','','',''], authors=set(), 
                  depth=0, max_depth=0, root='', sub=''):
    thread = []
    if item['author']=='[deleted]' or pycld2.detect(remove_bad_chars(item['body']))[2][0][1] not in langs:
        return [], 0
    
    authors.add(item['author'])
    if len(authors) > 2: 
        return [], 0
    
    current = [item[k] for k in ['body','author','id','created_utc','controversiality','score']]
    if 'author_flair_css_class' in item: current.extend([item['author_flair_css_class'],
                                                         item['author_flair_text']])
    else: current.extend(['',''])
    
    max_depth = depth
    collapse = []
    for item_i in item['children']:
        if item_i['id'] == item['id']: depth_i = depth
        else: depth_i = depth+1
        prse, max_depth_i = parse_comment(item_i, parent=current[1:8], authors=authors,
                                          depth=depth_i, max_depth=depth_i, root=root, sub=sub)
        
        if prse == 'DELETE': return 'DELETE', 0
        if len(prse)>0 and prse[-1][1] == item['author']: 
            collapse.append(prse[-1])
            prse = prse[:-1]
            
        max_depth = max(max_depth_i, max_depth)
        thread.extend(prse)

    current.extend([depth, max_depth, sub, root])
    current.extend(parent)
    for item_i in collapse:
        current[0] += item_i[0]
        current[2] = item_i[2]
        thread.append(current)
    if len(collapse)==0: thread.append(current)
    return thread, max_depth

def save_file_worker(input_file,output_file):
    with gzip.open(input_file) as f:
        ind = 0
        user_subreddit = []
        counter = 0
        all_threads = []
        for line in f:
            try:
                j = json.loads(line)
                thread, _ = parse_comment(j, parent=['','','','','','',''], authors=set(),
                                          depth=0, root=j['id'], sub=j['subreddit'])
                if thread != "DELETE": 
                    if len(thread) > 2: 
                        all_threads.extend(thread)
                ind += 1
                #if ind > 50000: break
                if ind % 500000 == 0:
                    user_subreddit.append(build_df(all_threads, output_file, counter))
                    all_threads = []
                    counter += 1
            except BaseException as e:
                print(output_file, e)
                ind += 0

        try:
            user_subreddit.append(build_df(all_threads, output_file, counter))
            pd.concat(user_subreddit).groupby(['subreddit','author'])['comment_id'].sum().reset_index().\
                to_csv(output_file+'.user_subreddit.csv', index=None)
        except Exception as e:
            print(output_file, e)
            ind += 1
    return 



In [7]:
# TEST

base_dir='data/conversational-reddit-comment-trees/'
fname = os.listdir(base_dir)[10]
fname = 'RC_2022-12.min-3-depth.gz'
input_file = base_dir + fname
output_file = 'data/conversation-turns-tokenized-aba/' + fname.split('.')[0]


In [8]:
print(datetime.now())
with gzip.open(input_file) as f:
    ind = 0
    user_subreddit = []
    counter = 0
    all_threads = []
    for line in f:
        j = json.loads(line)
        thread, _ = parse_comment(j, parent=['','','','',''], authors=set(),
                                  depth=0, root=j['id'], sub=j['subreddit'])
        if thread != "DELETE": 
            if len(thread) > 2: 
                all_threads.extend(thread)
        ind += 1
        #if ind > 50000: break
        if ind % 5000 == 0:
            user_subreddit.append(build_df(all_threads, output_file, counter))
            all_threads = []
            counter += 1
            if counter > 1: break

    pd.concat(user_subreddit).groupby(['subreddit','author'])['comment_id'].sum().reset_index().\
        to_csv(output_file+'.user_subreddit.csv', index=None)
print(datetime.now())


2023-04-04 17:51:13.344098
2023-04-04 17:51:17.218924


In [ ]:
save_file_worker(input_file,output_file)

In [10]:
def save_file(mode):
    pool = Pool(15)
    if mode=='thread_aba':
        inputs = []
        base_dir='data/conversational-reddit-comment-trees'
        save_dir='data/conversation-turns-tokenized-aba/'
        for file in sorted(os.listdir(base_dir)):
            if (not file.startswith("RC_2018") and file.endswith(".gz")):
                inputs.append((
                    join(base_dir,file),
                    join(save_dir,file.split('.')[0])
                ))
        # save_file_worker(*inputs[0])
        pool.starmap(save_file_worker,inputs)
    return

if __name__=='__main__':
    print(datetime.now())
    save_file(mode='thread_aba')
    print(datetime.now())


2023-04-04 17:53:49.982012
data/conversation-turns-tokenized-aba/RC_2019-08 input contains invalid UTF-8 around byte 269 (of 325)
data/conversation-turns-tokenized-aba/RC_2019-08 input contains invalid UTF-8 around byte 1537 (of 5938)
data/conversation-turns-tokenized-aba/RC_2019-07 input contains invalid UTF-8 around byte 289 (of 2185)
data/conversation-turns-tokenized-aba/RC_2019-07 Cannot take a larger sample than population when 'replace=False'
data/conversation-turns-tokenized-aba/RC_2020-02 Cannot take a larger sample than population when 'replace=False'
data/conversation-turns-tokenized-aba/RC_2019-08 Cannot take a larger sample than population when 'replace=False'
data/conversation-turns-tokenized-aba/RC_2021-06 input contains invalid UTF-8 around byte 1699 (of 2382)
data/conversation-turns-tokenized-aba/RC_2021-04 input contains invalid UTF-8 around byte 4 (of 8)
data/conversation-turns-tokenized-aba/RC_2021-11 input contains invalid UTF-8 around byte 5469 (of 6404)
data/conve

## Random Sample

In [ ]:
import pickle as pkl
for y in range(2019,2020):
    df = []
    for f in sorted(os.listdir('data/conversation-turns-tokenized-random/')):
        if f.startswith('RC_' + str(y)):
            print(datetime.now(), f)
            df.append(pkl.load(open('data/conversation-turns-tokenized-random/'+f, 'rb')))
    df = pd.concat(df)
    df = df.reset_index()
    print(df.shape)
    pkl.dump(df, open('data/conversation-turns-tokenized-random/full_sample_'+str(y)+'.pkl', 'wb'))
    

In [16]:
df = []
df_all = []
for y in range(2019,2023):
    print(datetime.now(), y)
    temp = pkl.load(open('data/conversation-turns-tokenized-random/full_sample_'+str(y)+'.pkl', 'rb'))
    temp['len_a'] = temp['tokens_liwc_parent'].apply(lambda x: x['length'] 
                                                     if type(x)!=float else 0)
    temp['len_b'] = temp['tokens_liwc'].apply(lambda x: x['length'])
    temp = temp[(temp['parent_id']!='') & (temp['len_a']>0) & (temp['len_b']>0)]
    print(temp.shape)
    df_all.append(temp)
    num_rows = 250000
    if y > 2019 and y < 2022: num_rows = 500000
    temp = temp.sample(n = num_rows)
    df.append(temp)
    del temp
df = pd.concat(df)
df = df.reset_index()
print(df.shape)
pkl.dump(df, open('data/conversation-turns-tokenized-random/full_sample.pkl', 'wb'))
    

2023-04-16 11:51:00.149775 2019
(8588559, 27)
2023-04-16 11:58:57.368783 2020
(19516158, 27)
2023-04-16 12:27:36.688529 2021
(21571145, 27)
2023-04-16 12:56:53.844679 2022
(21206733, 27)
(1500000, 28)


In [14]:
pkl.dump(df, open('data/conversation-turns-tokenized-random/full_sample.pkl', 'wb'))


In [13]:
df.to_csv('data/conversation-turns-tokenized-random/full_sample.pkl', index=None)


In [5]:
formality = pd.DataFrame(pkl.load(open('data/dl-inferred-style-scores/all_scores.formality.pkl', 'rb')))
offensive = pd.DataFrame(pkl.load(open('data/dl-inferred-style-scores/all_scores.offensive.pkl', 'rb')))
ids_f = set(formality['column_id'])
ids_o = set(offensive['column_id'])
ids_both = ids_o.intersection(ids_f)
del formality, offensiveness

In [41]:
def save_file_worker(y):

    dfs_formality = []
    dfs_offensive = []
    dfs_both = []

    files = sorted(os.listdir('data/conversation-turns-tokenized-aba/'))
    files = [f for f in files if f.startswith('RC_'+str(y)) and f.endswith('.pkl')]
    for f in files:
        try:
            print(datetime.now(), f)
            df = pkl.load(open('/shared/3/projects/style-influence/data/conversation-turns-tokenized-aba/'+f,'rb'))
            df = df[[c for c in df.columns 
                     if c not in ['tokens_liwc','tokens_liwc_parent']]]
            dfs_formality.append(df[df.comment_id.isin(ids_f)])
            dfs_offensive.append(df[df.comment_id.isin(ids_o)])
            dfs_both.append(df[df.comment_id.isin(ids_both)])
        except e as BaseException:
            print(f, e)            

    dfs_formality = pd.concat(dfs_formality)
    dfs_formality.to_csv('data/final_samples/formality_'+str(y)+'.tsv', sep='\t', index=None)

    dfs_offensive = pd.concat(dfs_offensive)
    dfs_offensive.to_csv('data/final_samples/offensive_'+str(y)+'.tsv', sep='\t', index=None)

    dfs_both = pd.concat(dfs_both)
    dfs_both.to_csv('data/final_samples/full_metrics_'+str(y)+'.tsv', sep='\t', index=None)

    return


In [42]:
pool = Pool(4)
pool.map(save_file_worker, range(2019,2023))


2023-04-21 11:15:48.134252 RC_2019-06-00.pkl
2023-04-21 11:15:48.1371402023-04-21 11:15:48.1372102023-04-21 11:15:48.138636   RC_2020-01-00.pklRC_2022-01-00.pklRC_2021-01-00.pkl


2023-04-21 11:16:36.183841 RC_2019-06-01.pkl
2023-04-21 11:16:41.844330 RC_2021-01-01.pkl
2023-04-21 11:16:48.447825 RC_2020-01-01.pkl
2023-04-21 11:16:49.311796 RC_2022-01-01.pkl
2023-04-21 11:17:12.353476 RC_2019-06-02.pkl
2023-04-21 11:17:22.960517 RC_2021-01-02.pkl
2023-04-21 11:17:31.647303 RC_2020-01-02.pkl
2023-04-21 11:17:37.760756 RC_2022-01-02.pkl
2023-04-21 11:17:56.549088 RC_2019-06-03.pkl
2023-04-21 11:18:08.067123 RC_2020-01-03.pkl
2023-04-21 11:18:09.518386 RC_2021-01-03.pkl
2023-04-21 11:18:27.718660 RC_2022-01-03.pkl
2023-04-21 11:18:29.252518 RC_2019-06-04.pkl
2023-04-21 11:18:50.103716 RC_2021-01-04.pkl
2023-04-21 11:18:50.111271 RC_2020-01-04.pkl
2023-04-21 11:19:08.790355 RC_2019-06-05.pkl
2023-04-21 11:19:10.200309 RC_2022-01-04.pkl
2023-04-21 11:19:29.045752 RC_2021-01-05.pkl
2023-04-21

2023-04-21 12:25:09.865120 RC_2020-05-06.pkl
2023-04-21 12:25:12.752909 RC_2019-11-12.pkl
2023-04-21 12:25:35.930723 RC_2021-04-18.pkl
2023-04-21 12:25:36.747913 RC_2022-04-13.pkl
2023-04-21 12:25:52.768641 RC_2020-05-07.pkl
2023-04-21 12:26:04.944931 RC_2019-11-13.pkl
2023-04-21 12:26:31.854928 RC_2022-04-14.pkl
2023-04-21 12:26:47.168069 RC_2020-05-08.pkl
2023-04-21 12:26:48.746628 RC_2021-04-19.pkl
2023-04-21 12:26:49.095538 RC_2019-11-14.pkl
2023-04-21 12:27:24.267873 RC_2022-04-15.pkl
2023-04-21 12:27:27.895189 RC_2021-04-20.pkl
2023-04-21 12:27:28.500825 RC_2020-05-09.pkl
2023-04-21 12:27:32.119778 RC_2019-11-15.pkl
2023-04-21 12:28:15.622045 RC_2019-11-16.pkl
2023-04-21 12:28:15.815011 RC_2022-04-16.pkl
2023-04-21 12:28:16.941220 RC_2021-04-21.pkl
2023-04-21 12:28:23.954687 RC_2020-05-10.pkl
2023-04-21 12:28:56.700406 RC_2022-04-17.pkl
2023-04-21 12:29:00.891330 RC_2019-12-00.pkl
2023-04-21 12:29:05.131844 RC_2020-05-11.pkl
2023-04-21 12:29:13.689199 RC_2021-04-22.pkl
2023-04-21

2023-04-21 14:04:37.072735 RC_2020-09-21.pkl
2023-04-21 14:04:51.247914 RC_2022-08-24.pkl
2023-04-21 14:05:00.427362 RC_2021-09-11.pkl
2023-04-21 14:05:02.171040 RC_2020-10-00.pkl
2023-04-21 14:05:46.683144 RC_2022-08-25.pkl
2023-04-21 14:05:50.154547 RC_2020-10-01.pkl
2023-04-21 14:06:10.327046 RC_2021-09-12.pkl
2023-04-21 14:06:37.954434 RC_2020-10-02.pkl
2023-04-21 14:06:38.612612 RC_2022-08-26.pkl
2023-04-21 14:07:20.283584 RC_2021-09-13.pkl
2023-04-21 14:07:26.734276 RC_2022-08-27.pkl
2023-04-21 14:07:29.749522 RC_2020-10-03.pkl
2023-04-21 14:07:47.347116 RC_2022-10-00.pkl
2023-04-21 14:08:14.213861 RC_2021-09-14.pkl
2023-04-21 14:08:19.537513 RC_2020-10-04.pkl
2023-04-21 14:08:31.680492 RC_2022-10-01.pkl
2023-04-21 14:09:08.072567 RC_2020-10-05.pkl
2023-04-21 14:09:10.257267 RC_2021-09-15.pkl
2023-04-21 14:09:33.927233 RC_2022-10-02.pkl
2023-04-21 14:09:54.837274 RC_2020-10-06.pkl
2023-04-21 14:10:26.604703 RC_2021-09-16.pkl
2023-04-21 14:10:33.216735 RC_2022-10-03.pkl
2023-04-21

[None, None, None, None]